In [ ]:
import jieba
import re
import sys ,os
jieba.suggest_freq('韓國瑜', True)
jieba.suggest_freq('兩岸一家親' , True)


sys.setrecursionlimit(1000000)
PUNC = "[0-9a-zA-Z\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）+「」，。）（〔〕／『』:\[\]’：\-《》％‘○]"

text = '陳菊與美國國務院亞太事務代理助理國務卿董雲裳（右）會面。（劉世芳提供）〔記者葛祐豪／高雄報導〕高雄市長陳菊前往美國訪問，今天與美國國務院亞太事務代理助理國務卿董雲裳等人會面；董雲裳強調，台美友誼深厚，也是堅強的夥伴。陳菊在立委劉世芳、賴瑞隆陪同下，今天與美國國務院亞太事務代理助卿董雲裳（Susan Thornton）、經濟暨商業事務首席副助卿麥費特（Brian McFeeters）、亞太事務代理副助卿史墨客（Hanscom Smith）會面。董雲裳回憶說，2000年駐台期間，她在台安醫院生下小女兒，當時父母曾來台探望。前幾天她翻出家人一起環島旅行的相片，對台灣充滿著思念。陳菊告訴董雲裳說，這次訪美主要是到CSIS智庫演講，分享高雄的城市轉型經驗及台灣的民主轉型經驗。陳菊說，她的演講也提到蔡英文總統在內政方面啟動的改革很重要，非成功不可。在兩岸方面，蔡英文溫和、穩健、對中國保持善意，但中國卻沒有正面的回應。她支持蔡總統的政策，也希望美方給予支持。董雲裳表示，台美雙方有互信，更有共同的自由、民主、人權價值，她看到蔡總統戮力改革，積極讓全世界聽到台灣人的聲音。她完全了解改革的困難，也很欽佩蔡總統推動改革的視野。董雲裳認為，台灣只有2300萬人口，卻是美國第11大貿易夥伴。台灣對美國非常重要。台美在安全方面的合作，今後只會更加強，至於台灣參與國際組織，美國一貫支持，也將積極協助提高台灣的國際能見度。副助卿麥費特提醒，美台經濟合作的三大重點：（1）希望美國公司能夠在台公平投資、公平銷售；（2）保護智慧財產權（3）加強高層技術（物聯網、AI等）的合作。董雲裳也提到，北京對台灣的壓力日漸增加，應謀對策，而美國與台灣同為民主國家，應該站在同一陣線。最後，陳菊贈送董雲裳一條排灣族的琉璃珠手鍊。這條「眼睛之珠」，傳說有守護的功用。董雲裳戴上後說，她已經感覺到被保護了。立委劉世芳陪同陳菊與美國國務院亞太事務代理助卿董雲裳（Susan Thornton）等人會面。（劉世芳提供'

In [220]:
text = re.sub(PUNC , "",text)

In [221]:
text_list = list(jieba.cut(text))

In [235]:
def print_rule(my_dict , rules):
    for k,v in rules.items():
        for kk in k:
            print(my_dict.INV_WORD_IDX[kk],end=' ')
        print('->',end=' ')
        for vv in v[0]:
            print(my_dict.INV_WORD_IDX[vv],end=' ')
        print(v[-1])
        
def print_frp(my_dict , patterns):
    for k,v in patterns.items():
        if(len(k) < 2):
            continue
        for kk in k:
            print(my_dict.INV_WORD_IDX[kk],end=' ')
        print(v)

In [255]:
import pickle
from math import log
class shared_dict(object):
    def __init__(self): 
        self.WORD_IDX = {}
        self.INV_WORD_IDX = {}
        self.DF = {}
        self.count = 0       
        self.total_doc = 0 
        self.PUNC = "[0-9a-zA-Z\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）+「」，。）（〔〕／『』:\[\]’‘：\-《》％○]"

    
    def add_word(self , word):
        if word in self.WORD_IDX: 
            pass        
        else:
            self.WORD_IDX[word] = self.count
            self.INV_WORD_IDX[self.count] = word
            self.count = self.count + 1
            
    def add_doc(self , doc):
    
        self.total_doc += 1
        word_exist = set()
        
        for word in doc:
            word_exist.add(word)
            self.add_word(word)
            
        for word in word_exist:
            idx = self.WORD_IDX[word]
            if idx in self.DF:
                self.DF[idx] += 1
            else:
                self.DF[idx] = 1
        
    def get_seq(self , word_list , TOPK = 55):
        seq = []
        TF = {}
        tf_idf = {}
        
        total_word = len(word_list)
        
        for word in word_list:
            idx = self.WORD_IDX[word]
            
            if idx in TF:
                TF[idx] += 1
            else:
                TF[idx] = 1
                
        for word in word_list:
            idx = self.WORD_IDX[word]
            tf_idf[idx] = (TF[idx] / total_word) * log( self.total_doc / (self.DF[idx]+1) )
                
                
        sorted_word = list(reversed(sorted(tf_idf.items() , key = lambda d : d[1])))
                
        keep_word = []
                
        for k,v in sorted_word:
            keep_word.append(k)

            if(len(sorted_word) > TOPK):
                keep_word = keep_word[:TOPK]
                        
        return keep_word
                    
    
    def get_transcation_db(self,lines):
        
        # lines should be list of text list
        transcations = []
        for text_list in lines:
            
            seq = self.get_seq(text_list)
            transcations.append(seq)
        
        return transcations
            
    def save_dict(self):
        out_f = open('word_index.pickle','wb')
                    
        pickle.dump(self.WORD_IDX , out_f)
        
        out_f.close()
        
    def load_dict(self):
        in_f = open('word_index.pickle','rb')
        self.WORD_IDX = pickle.load(in_f)
        in_f.close()

        for word , idx in self.WORD_IDX.items():
            self.INV_WORD_IDX[idx] = word



In [263]:
cur_keyword = '張安樂'

In [264]:
ct_dict = shared_dict()
dir_name = cur_keyword+'_中時'
f_names = os.listdir(dir_name)
text_lists = []
transcations = []

for f_name in f_names:
    f = open(dir_name+'/'+f_name,'r',encoding='utf-8')
    lines = f.readlines()
    
    
    for text in lines:
        text = re.sub(PUNC , "" , text)
        text_list = list( jieba.cut(text) )
        text_lists.append(text_list)
        ct_dict.add_doc(text_list)

ct_transcations = ct_dict.get_transcation_db(text_lists)


In [265]:
ltn_dict = shared_dict()
dir_name = cur_keyword+'_自由'
f_names = os.listdir(dir_name)
text_lists = []
transcations = []

for f_name in f_names:
    f = open(dir_name+'/'+f_name,'r',encoding='utf-8')
    lines = f.readlines()

    
    for text in lines:
        text = re.sub(PUNC , "" , text)
        text_list = list( jieba.cut(text) )
        text_lists.append(text_list)
        ltn_dict.add_doc(text_list)

ltn_transcations = ltn_dict.get_transcation_db(text_lists)


In [275]:
ct_num = len(ct_transcations) 
ltn_num = len(ltn_transcations)
ct_num,ltn_num

(28, 41)

In [287]:
thold = 0.01
ct_thold = max(3 , int ( ct_num * thold ) )
ltn_thold = max(3 , int( ltn_num * thold ) )
ct_thold,ltn_thold

(3, 3)

In [ ]:
%%time
import pyfpgrowth
ltn_patterns = pyfpgrowth.find_frequent_patterns(ltn_transcations, ltn_thold)
print('ltn_done')
ct_patterns = pyfpgrowth.find_frequent_patterns(ct_transcations,ct_thold)
print('ct_done')

ltn_done


In [281]:
print_frp(ct_dict , ct_patterns)

他 當年 4
汪希苓 當年 4
江南案 當年 4
蔣經國 當年 4
他 汪希苓 當年 4
他 江南案 當年 4
他 蔣經國 當年 4
江南案 汪希苓 當年 4
汪希苓 蔣經國 當年 4
江南案 蔣經國 當年 4
他 江南案 汪希苓 當年 4
他 汪希苓 蔣經國 當年 4
他 江南案 蔣經國 當年 4
江南案 汪希苓 蔣經國 當年 4
他 江南案 汪希苓 蔣經國 當年 4
他 關係 4
汪希苓 關係 4
江南案 關係 4
蔣經國 關係 4
他 汪希苓 關係 4
他 江南案 關係 4
他 蔣經國 關係 4
江南案 汪希苓 關係 4
汪希苓 蔣經國 關係 4
江南案 蔣經國 關係 4
他 江南案 汪希苓 關係 4
他 汪希苓 蔣經國 關係 4
他 江南案 蔣經國 關係 4
江南案 汪希苓 蔣經國 關係 4
他 江南案 汪希苓 蔣經國 關係 4
她 下 4
台灣 新館 4
美國 新館 4
台灣 美國 4
台灣 美國 新館 4
頒發 家長 4
市長獎 家長 4
江南案 汪希苓 4
汪希苓 蔣經國 4
江南案 汪希苓 蔣經國 4
江南案 蔣經國 5
他 汪希苓 4
他 江南案 4
他 蔣經國 4
他 江南案 汪希苓 4
他 汪希苓 蔣經國 4
他 江南案 蔣經國 4
他 江南案 汪希苓 蔣經國 4
她 璩美鳳 4


In [282]:
print_frp(ltn_dict , ltn_patterns)

為了 你 3
警方 發現 3
沖繩 旭琉會 3
日本 旭琉會 3
日本 沖繩 旭琉會 3
前往 旭琉會 3
前往 沖繩 3
前往 日本 3
前往 沖繩 旭琉會 3
前往 日本 旭琉會 3
前往 日本 沖繩 3
前往 日本 沖繩 旭琉會 3
沖繩 張瑋 3
日本 張瑋 3
日本 沖繩 張瑋 3
幹部 警方 3
臉書 網友 3
帶 聯盟 3
聯盟 年改 3
聯盟 繆 年改 3
帶 聯盟 繆 3
聯盟 繆 德生 3
聯盟 德生 年改 3
聯盟 繆 德生 年改 3
帶 聯盟 德生 3
帶 聯盟 繆 德生 3
聯盟 德生 五星旗 3
聯盟 繆 五星旗 3
聯盟 繆 德生 五星旗 3
聯盟 年改 五星旗 3
聯盟 繆 年改 五星旗 3
聯盟 德生 年改 五星旗 3
聯盟 繆 德生 年改 五星旗 3
帶 聯盟 五星旗 3
帶 聯盟 繆 五星旗 3
帶 聯盟 德生 五星旗 3
帶 聯盟 繆 德生 五星旗 3
藍天 聯盟 3
帶 藍天 3
帶 藍天 聯盟 3
藍天 年改 3
藍天 聯盟 年改 3
藍天 繆 年改 3
帶 藍天 繆 3
帶 藍天 聯盟 繆 3
藍天 聯盟 繆 年改 3
藍天 繆 德生 3
藍天 德生 年改 3
藍天 繆 德生 年改 3
帶 藍天 德生 3
帶 藍天 繆 德生 3
帶 藍天 聯盟 德生 3
藍天 聯盟 德生 年改 3
藍天 聯盟 繆 德生 年改 3
帶 藍天 聯盟 繆 德生 3
藍天 德生 五星旗 3
藍天 繆 五星旗 3
藍天 繆 德生 五星旗 3
藍天 年改 五星旗 3
藍天 繆 年改 五星旗 3
藍天 德生 年改 五星旗 3
藍天 繆 德生 年改 五星旗 3
帶 藍天 五星旗 3
帶 藍天 繆 五星旗 3
帶 藍天 德生 五星旗 3
帶 藍天 繆 德生 五星旗 3
帶 藍天 聯盟 五星旗 3
藍天 聯盟 年改 五星旗 3
藍天 聯盟 繆 年改 五星旗 3
帶 藍天 聯盟 繆 五星旗 3
藍天 聯盟 繆 德生 五星旗 3
藍天 聯盟 德生 年改 五星旗 3
藍天 聯盟 繆 德生 年改 五星旗 3
帶 藍天 聯盟 德生 五星旗 3
帶 藍天 聯盟 繆 德生 五星旗 3
年改 衝突 3
繆 衝突 3
德生 衝突 3
繆 德生 衝突 3
五星旗 衝突 3
德生 五星旗 衝突 3
繆 五星旗 衝突 3
繆 德生 五星旗 衝突 3
反 年改 3
反 繆 3
反 

大學 邀請 教育 附設 3
大學 邀請 教育 畢業生 3
邀請 學校 國立 教育 上台 3
邀請 學校 國立 教育 附設 3
邀請 學校 國立 教育 畢業生 3
大學 邀請 學校 國立 教育 3
邀請 國立 教育 附設 上台 3
邀請 國立 教育 上台 畢業生 3
大學 邀請 國立 教育 上台 3
邀請 國立 教育 附設 畢業生 3
大學 邀請 國立 教育 附設 3
大學 邀請 國立 教育 畢業生 3
邀請 學校 教育 附設 上台 3
邀請 學校 教育 上台 畢業生 3
大學 邀請 學校 教育 上台 3
邀請 學校 教育 附設 畢業生 3
大學 邀請 學校 教育 附設 3
大學 邀請 學校 教育 畢業生 3
邀請 教育 附設 上台 畢業生 3
大學 邀請 教育 附設 上台 3
大學 邀請 教育 上台 畢業生 3
大學 邀請 教育 附設 畢業生 3
邀請 學校 國立 教育 附設 上台 3
邀請 學校 國立 教育 上台 畢業生 3
大學 邀請 學校 國立 教育 上台 3
邀請 學校 國立 教育 附設 畢業生 3
大學 邀請 學校 國立 教育 附設 3
大學 邀請 學校 國立 教育 畢業生 3
邀請 國立 教育 附設 上台 畢業生 3
大學 邀請 國立 教育 附設 上台 3
大學 邀請 國立 教育 上台 畢業生 3
大學 邀請 國立 教育 附設 畢業生 3
邀請 學校 教育 附設 上台 畢業生 3
大學 邀請 學校 教育 附設 上台 3
大學 邀請 學校 教育 上台 畢業生 3
大學 邀請 學校 教育 附設 畢業生 3
大學 邀請 教育 附設 上台 畢業生 3
邀請 學校 國立 教育 附設 上台 畢業生 3
大學 邀請 學校 國立 教育 附設 上台 3
大學 邀請 學校 國立 教育 上台 畢業生 3
大學 邀請 學校 國立 教育 附設 畢業生 3
大學 邀請 國立 教育 附設 上台 畢業生 3
大學 邀請 學校 教育 附設 上台 畢業生 3
大學 邀請 學校 國立 教育 附設 上台 畢業生 3
邀請 頒發 3
邀請 教育 頒發 3
邀請 國立 頒發 3
邀請 學校 頒發 3
邀請 上台 頒發 3
邀請 附設 頒發 3
邀請 頒發 畢業生 3
大學 邀請 頒發 3
邀請 國立 教育 頒發 3
邀請 學校 教育 頒發 3
邀請 教育 上台 頒發 3
邀請 教育 附設 頒發 3

邀請 學校 國立 附設 上台 頒發 市長獎 畢業生 3
大學 邀請 學校 國立 附設 上台 頒發 市長獎 3
大學 邀請 學校 國立 上台 頒發 市長獎 畢業生 3
大學 邀請 學校 國立 附設 頒發 市長獎 畢業生 3
大學 邀請 國立 附設 上台 頒發 市長獎 畢業生 3
大學 邀請 學校 附設 上台 頒發 市長獎 畢業生 3
邀請 學校 國立 教育 附設 上台 市長獎 畢業生 3
大學 邀請 學校 國立 教育 附設 上台 市長獎 3
大學 邀請 學校 國立 教育 上台 市長獎 畢業生 3
大學 邀請 學校 國立 教育 附設 市長獎 畢業生 3
大學 邀請 國立 教育 附設 上台 市長獎 畢業生 3
大學 邀請 學校 教育 附設 上台 市長獎 畢業生 3
大學 邀請 學校 國立 附設 上台 市長獎 畢業生 3
邀請 學校 國立 教育 附設 上台 頒發 市長獎 畢業生 3
大學 邀請 學校 國立 教育 附設 上台 頒發 市長獎 3
大學 邀請 學校 國立 教育 上台 頒發 市長獎 畢業生 3
大學 邀請 學校 國立 教育 附設 頒發 市長獎 畢業生 3
大學 邀請 國立 教育 附設 上台 頒發 市長獎 畢業生 3
大學 邀請 學校 教育 附設 上台 頒發 市長獎 畢業生 3
大學 邀請 學校 國立 附設 上台 頒發 市長獎 畢業生 3
大學 邀請 學校 國立 教育 附設 上台 市長獎 畢業生 3
大學 邀請 學校 國立 教育 附設 上台 頒發 市長獎 畢業生 3
大學 邀請 4
這 頒獎 3
個 頒獎 3
個 畢業典禮 3
國北教大 請辭 3
頒獎 國北教大 請辭 3
校方 請辭 3
校方 頒獎 請辭 3
上台 請辭 3
上台 頒獎 請辭 3
畢業典禮 請辭 3
畢業典禮 上台 請辭 3
畢業典禮 頒獎 請辭 3
畢業典禮 上台 頒獎 請辭 3
家長 請辭 3
校方 家長 請辭 3
家長 頒獎 請辭 3
校方 家長 頒獎 請辭 3
市長獎 請辭 3
市長獎 國北教大 請辭 3
市長獎 頒獎 請辭 3
市長獎 頒獎 國北教大 請辭 3
頒獎 請辭 4
校長 國北教大 3
校長 頒獎 國北教大 3
校長 家長 國北教大 3
校長 校方 國北教大 3
校長 家長 頒獎 國北教大 3
校長 校方 頒獎 國北教大 3
校長 校方 家長 國北教大 3
校長 校方 家長 頒獎 

教育 附設 上台 頒發 市長獎 家長 頒獎 3
教育 附設 上台 頒發 市長獎 家長 典禮 頒獎 3
學校 教育 附設 頒發 家長 頒獎 3
學校 教育 附設 頒發 家長 典禮 頒獎 3
學校 教育 附設 頒發 市長獎 家長 頒獎 3
學校 教育 附設 頒發 市長獎 家長 典禮 頒獎 3
國立 教育 附設 家長 3
國立 附設 家長 頒獎 3
國立 教育 附設 家長 頒獎 3
國立 附設 市長獎 家長 3
國立 附設 市長獎 家長 頒獎 3
國立 教育 附設 市長獎 家長 3
國立 教育 附設 市長獎 家長 頒獎 3
國立 附設 家長 典禮 3
國立 附設 市長獎 家長 典禮 3
國立 附設 家長 典禮 頒獎 3
國立 教育 附設 家長 典禮 3
國立 附設 市長獎 家長 典禮 頒獎 3
國立 教育 附設 市長獎 家長 典禮 3
國立 教育 附設 家長 典禮 頒獎 3
國立 教育 附設 市長獎 家長 典禮 頒獎 3
國立 附設 上台 家長 3
國立 附設 上台 家長 典禮 3
國立 附設 上台 市長獎 家長 3
國立 附設 上台 家長 頒獎 3
國立 教育 附設 上台 家長 3
國立 附設 上台 市長獎 家長 典禮 3
國立 附設 上台 家長 典禮 頒獎 3
國立 教育 附設 上台 家長 典禮 3
國立 附設 上台 市長獎 家長 頒獎 3
國立 教育 附設 上台 市長獎 家長 3
國立 教育 附設 上台 家長 頒獎 3
國立 附設 上台 市長獎 家長 典禮 頒獎 3
國立 教育 附設 上台 市長獎 家長 典禮 3
國立 教育 附設 上台 家長 典禮 頒獎 3
國立 教育 附設 上台 市長獎 家長 頒獎 3
國立 教育 附設 上台 市長獎 家長 典禮 頒獎 3
學校 國立 附設 家長 3
學校 國立 附設 家長 典禮 3
學校 國立 附設 市長獎 家長 3
學校 國立 附設 家長 頒獎 3
學校 國立 教育 附設 家長 3
學校 國立 附設 市長獎 家長 典禮 3
學校 國立 附設 家長 典禮 頒獎 3
學校 國立 教育 附設 家長 典禮 3
學校 國立 附設 市長獎 家長 頒獎 3
學校 國立 教育 附設 市長獎 家長 3
學校 國立 教育 附設 家長 頒獎 3
學校 國立 附設 市長獎 家長 典禮 頒獎 3
學校 國立 教育 附設 市長獎 家長 典禮 3
學校 國

學校 國立 教育 市長獎 家長 典禮 畢業生 3
學校 國立 頒發 市長獎 家長 典禮 畢業生 3
學校 國立 市長獎 家長 典禮 畢業生 頒獎 3
學校 教育 頒發 市長獎 家長 典禮 畢業生 3
學校 教育 市長獎 家長 典禮 畢業生 頒獎 3
學校 頒發 市長獎 家長 典禮 畢業生 頒獎 3
學校 國立 教育 頒發 市長獎 家長 典禮 畢業生 3
學校 國立 教育 市長獎 家長 典禮 畢業生 頒獎 3
學校 國立 頒發 市長獎 家長 典禮 畢業生 頒獎 3
學校 教育 頒發 市長獎 家長 典禮 畢業生 頒獎 3
學校 國立 教育 頒發 市長獎 家長 典禮 畢業生 頒獎 3
上台 市長獎 家長 典禮 畢業生 3
國立 上台 市長獎 家長 典禮 畢業生 3
教育 上台 市長獎 家長 典禮 畢業生 3
上台 頒發 市長獎 家長 典禮 畢業生 3
上台 市長獎 家長 典禮 畢業生 頒獎 3
國立 教育 上台 市長獎 家長 典禮 畢業生 3
國立 上台 頒發 市長獎 家長 典禮 畢業生 3
國立 上台 市長獎 家長 典禮 畢業生 頒獎 3
教育 上台 頒發 市長獎 家長 典禮 畢業生 3
教育 上台 市長獎 家長 典禮 畢業生 頒獎 3
上台 頒發 市長獎 家長 典禮 畢業生 頒獎 3
國立 教育 上台 頒發 市長獎 家長 典禮 畢業生 3
國立 教育 上台 市長獎 家長 典禮 畢業生 頒獎 3
國立 上台 頒發 市長獎 家長 典禮 畢業生 頒獎 3
教育 上台 頒發 市長獎 家長 典禮 畢業生 頒獎 3
國立 教育 上台 頒發 市長獎 家長 典禮 畢業生 頒獎 3
上台 畢業生 4
上台 市長獎 畢業生 4
學校 畢業生 4
學校 上台 畢業生 3
學校 市長獎 畢業生 4
學校 上台 市長獎 畢業生 3
學校 國立 畢業生 4
國立 上台 畢業生 4
學校 國立 上台 畢業生 3
國立 上台 市長獎 畢業生 4
學校 國立 市長獎 畢業生 4
學校 國立 上台 市長獎 畢業生 3
國立 教育 畢業生 4
學校 教育 畢業生 4
學校 國立 教育 畢業生 4
教育 上台 畢業生 4
學校 教育 上台 畢業生 3
國立 教育 上台 畢業生 4
學校 國立 教育 上台 畢業生 3
教育 上台 市長獎 畢業生 4
學校 教育 市長獎 畢業生 4
學校 教育 上台 市長獎 畢

In [210]:
a = {
    'a':5,
    'b':6,
    'c':7,
    'd':1
}

In [159]:
zz = sorted(a.items() , key = lambda d : d[1])

In [160]:
for a,b in zz:
    print(a,b)

d 1
a 5
b 6
c 7
